In [1]:
import logging
import time
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geocoder
import requests
from urllib.request import urlopen, Request

# kemungkinan besar key token sudah expired
api_secret = '1C5y5mJ22wNRnYxiL7I3XxlWqw0dfZl1aoOfG58Td7JLHJiNCK'
api_key = 'zMl3pH44bPIkUkJoQAAqXw8WI'
bearer_token  = 'AAAAAAAAAAAAAAAAAAAAAFf9MwEAAAAA%2F5XsPkybcbam2e9i9lx8Xt5QMjI%3DRpph4jP9mu15X3REyoGMkgq7QbUGRQjla3djteWbwm8fwjuThq'
access_token = '1159389149368950784-hFHGhydaGXj7aADLrHrbT9goUA3rUI'
access_token_secret = 'QA1Cd2dgTCK7ZKn7f6V4sYTcFD7kzgcckObbPayLa5aBS'

auth = tweepy.OAuthHandler(api_key, api_secret); auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

Cara kerja chain network ini adalah dengan menghubungkan user dengan user lain berdasarkan aktivitas replies mereka.

Didasarkan dengan algoritma twitter yang selalu menampilkan aktivitas replies yang dilakukan suatu akun terhadap akun lain, maka secara otomatis akan terdiscover oleh pengikut akun yang mereply tersebut

Untuk melakukan community detection, kami mengikuti pola algoritma Netlytic (Who's replies whom) dan mengaplikasikannya dengan menggunakan advanced search bawaan Twitter API

Untuk penemuan kelompok komunitas akan dilakukan dan dideteksi oleh aplikasi Gephi

In [5]:
# mengambil tweet  yang muncul ditujukan pada akun utama (yang dicari)
def crawl_tweets(keywords, count, arr_tweet):
    search = keywords
    new_search = search
    list_tweet = arr_tweet
    tweets = tweepy.Cursor(api.search, q=new_search, lang="id").items(count)
    for tweet in tweets:
        if tweet.in_reply_to_status_id == None:
            continue
        else:
            list_tweet.append([tweet.user.screen_name, tweet.in_reply_to_screen_name])

    return list_tweet

# mengambil replies dari akun yang dicarikan
def crawl_replies(keywords, count, arr_tweet):
    search = keywords
    new_search = search
    list_tweet = arr_tweet
    idx = 0
    tweets = tweepy.Cursor(api.search, q=new_search, lang="id").items(count)
    for tweet in tweets:
        if tweet.in_reply_to_status_id == None:
            continue
        elif tweet.user.screen_name == tweet.in_reply_to_screen_name:
            continue
        else:
            list_tweet.append([tweet.user.screen_name, tweet.in_reply_to_screen_name])
        idx+=1
    
    return list_tweet

# berguna untuk mencari replies dari tiap replier
def search_replier(data,count):
    idx = 0
    for i in range(len(data)):
        try:
            user_screen_name = data[idx][0]
            query1 = '(to:'+user_screen_name+' filter:replies)'
            data = crawl_replies(query1,count,data)
            print(idx+1,'. ',query1)
            idx+=1
        except tweepy.RateLimitError as e:
            logging.error("Twitter api rate limit reached".format(e))
            time.sleep(60)
            continue
    return data

In [ ]:
# bila data sudah ada
data = pd.read_xlsx('namafile.xlsx')

In [6]:
# bila ingin inisialisasi data baru
data = []

In [8]:
# pengambilan data
data = crawl_tweets('(to:XiaomiIndonesia) min_replies:0 min_faves:0 min_retweets:0 lang:id', 75, data)

# disimpan menjadi excel
df = pd.DataFrame(data,columns=['from','to'])
df.to_csv('to_XiaomiIndonesia.csv', index=False)

In [9]:
data

[['ditakarina', 'XiaomiIndonesia'],
 ['KenJimmy88', 'XiaomiIndonesia'],
 ['MailMKD', 'XiaomiIndonesia'],
 ['remyjovi', 'XiaomiIndonesia'],
 ['Andrylino25', 'XiaomiIndonesia'],
 ['hawariyash', 'XiaomiIndonesia'],
 ['Dhenz507', 'XiaomiIndonesia'],
 ['adamramadhanrs', 'XiaomiIndonesia'],
 ['Yudi116515', 'XiaomiIndonesia'],
 ['MarajoSm', 'XiaomiIndonesia'],
 ['Didik16599789', 'XiaomiIndonesia'],
 ['UCL_Gamer_SMG', 'XiaomiIndonesia'],
 ['vellyfilliy', 'XiaomiIndonesia'],
 ['resnujr', 'XiaomiIndonesia'],
 ['DOCTOR_GLOCK_YT', 'XiaomiIndonesia'],
 ['Geraldhi_7D', 'XiaomiIndonesia'],
 ['ArrafifRafi', 'XiaomiIndonesia'],
 ['asalbyh', 'XiaomiIndonesia'],
 ['didihamzaa', 'XiaomiIndonesia'],
 ['leo_tech2broke', 'XiaomiIndonesia'],
 ['winteraphy', 'XiaomiIndonesia'],
 ['MuhKalam', 'XiaomiIndonesia'],
 ['fizahut_', 'XiaomiIndonesia'],
 ['FRiasati', 'XiaomiIndonesia'],
 ['asep_rifa', 'XiaomiIndonesia'],
 ['resnujr', 'XiaomiIndonesia'],
 ['kerjamuluk', 'XiaomiIndonesia'],
 ['Husen2801', 'XiaomiIndonesi

Karena keterbatasan API Twitter yang membatasi rate limit hanya 180 tweet per 15 menit

Akan dilakukan pengambilan data dengan beberapa kali attempt per tiap 15 menit

In [10]:
# pengambilan data attempt pertama
data_rep = search_replier(data,10)

# disimpan menjadi excel
df = pd.DataFrame(data_rep,columns=['from','to'])
df.to_csv('to_XiaomiIndonesia1.csv', index=False)

1 .  (to:ditakarina filter:replies)
2 .  (to:KenJimmy88 filter:replies)
3 .  (to:MailMKD filter:replies)
4 .  (to:remyjovi filter:replies)
5 .  (to:Andrylino25 filter:replies)
6 .  (to:hawariyash filter:replies)
7 .  (to:Dhenz507 filter:replies)
8 .  (to:adamramadhanrs filter:replies)
9 .  (to:Yudi116515 filter:replies)
10 .  (to:MarajoSm filter:replies)
11 .  (to:Didik16599789 filter:replies)
12 .  (to:UCL_Gamer_SMG filter:replies)
13 .  (to:vellyfilliy filter:replies)
14 .  (to:resnujr filter:replies)
15 .  (to:DOCTOR_GLOCK_YT filter:replies)
16 .  (to:Geraldhi_7D filter:replies)
17 .  (to:ArrafifRafi filter:replies)
18 .  (to:asalbyh filter:replies)
19 .  (to:didihamzaa filter:replies)
20 .  (to:leo_tech2broke filter:replies)
21 .  (to:winteraphy filter:replies)
22 .  (to:MuhKalam filter:replies)
23 .  (to:fizahut_ filter:replies)
24 .  (to:FRiasati filter:replies)
25 .  (to:asep_rifa filter:replies)
26 .  (to:resnujr filter:replies)
27 .  (to:kerjamuluk filter:replies)
28 .  (to:Hu

untuk percobaan kali ini dilakukan crawl hanya satu kali dikarenakan rate limit API Twitter

apabila ingin mendapatkan banyak nodes dan edges, dapat melakukan attempt kedua dan ketiga untuk hasil yang lebih dalam lagi (dengan waktu lebih lama)

In [ ]:
# tunggu 15 menit
# pengambilan data attempt kedua
# 39 disini adalah angka random
data_rep1 = search_replier(data_rep[39:],10)

# disimpan menjadi excel
df1 = pd.DataFrame(data_rep1,columns=['from','to'])
df1.to_csv('to_XiaomiIndonesia2.csv', index=False)

In [ ]:
# tunggu 15 menit
# pengambilan data attempt kedua
# 87 disini adalah angka random
data_rep1 = search_replier(data_rep1[87:],10)

# disimpan menjadi excel
df2 = pd.DataFrame(data_rep1,columns=['from','to'])
df2.to_csv('to_XiaomiIndonesia3.csv', index=False)

Kemudian data csv terakhir dapat diupload pada netlytic atau divisualisasikan pada gephi